In [11]:
import pandas as pd
import numpy as np
import os

# change working directory to project-2
if os.getcwd().split('/')[-1] != 'project-2':
    os.chdir('../../')

from src.utils.dnds_crude_utils import get_pval
from src.utils.eval  import evalAccuracy, compareRankings

# load the IntOGen ranking TSV
path_intogen = "data/raw/IntOGen-DriverGenes_TCGA_WXS_BRCA.tsv"
df_intogen = pd.read_csv(path_intogen, sep="\t")

# build a dict mapping gene names to their IntOGen relevance.
#RELEVANCE IS SAMPLES%
baseline_ranks = dict(zip(df_intogen["Symbol"], (df_intogen["Samples (%)"] * 0.01)))

In [12]:
# set paths
DNDS = 'results/tables/dnds_simple_results.tsv'
DNDSCV = 'results/tables/dNdScv_output.csv'

df_dnds = pd.read_csv(DNDS, sep='\t')
df_dndscv = pd.read_csv(DNDSCV)

df_dnds = get_pval(df_dnds)

In [18]:
df_dn_ds = df_dnds

dn_ds_ranks = dict(zip(df_dn_ds["Hugo_Symbol"], 1-df_dn_ds["fisher_pval"]))

# calculate accuracy metrics
dcg, bpref, accuracy = evalAccuracy(dn_ds_ranks, baseline_ranks)

df_gene_ranks = compareRankings(dn_ds_ranks, baseline_ranks)

print(f"dNdS: Using fisher pval, DCG: {dcg}, Bpref: {bpref}, Accuracy: {accuracy}")

obs = []
obs.append({'dcg': {dcg}, 'bpref': {bpref}, 'accuracy': {accuracy}})

dNdS: Using fisher pval, DCG: 0.6004291471392449, Bpref: 0.3983931947069944, Accuracy: 0.4130434782608695


In [19]:
df_dn_ds = df_dndscv

df_dn_ds['Hugo_Symbol'] = df_dn_ds['gene_name']

dn_ds_ranks = dict(zip(df_dn_ds["Hugo_Symbol"], 1-df_dn_ds["qglobal_cv"]))

# calculate accuracy metrics
dcg, bpref, accuracy = evalAccuracy(dn_ds_ranks, baseline_ranks)

df_gene_ranks = compareRankings(dn_ds_ranks, baseline_ranks)

print(f"dNdScv: Using qlobal_cv, DCG: {dcg}, Bpref: {bpref}, Accuracy: {accuracy}")
obs.append({'dcg': {dcg}, 'bpref': {bpref}, 'accuracy': {accuracy}})

dNdScv: Using qlobal_cv, DCG: 0.5920363154007605, Bpref: 0.5760869565217391, Accuracy: 0.5652173913043479


In [23]:
results = pd.DataFrame(obs, index=['dNdS','dNdScv'])
results.to_csv('results/tables/dnds_vs_dndscv.tsv', sep='\t')